In [1]:
from altProts_in_communities.utils import *
from altProts_in_communities.comppass_plus import *

In [2]:
BP_Filters_edges = set(frozenset((x['bait_symbol'], x['symbol'] )) for x in BP_noFilters if x['pInt']>0.75)
BP_neg = set(frozenset((x['bait_symbol'], x['symbol'] )) for x in BP_noFilters if x['pInt']<0.75)
HCIP_training_labels = BP_Filters_edges

In [3]:
synonyms = {
    'C3orf75':'ELP6', 
    'FUS|IP_2287811':'FUS|IP_243680'
}

In [4]:
features_table = []
with open('BP2_OP16_features.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for n, row in enumerate(reader):
        if n==0:
            cols = row
            cols[0] = 'Experiment_ID'
            continue
        line = dict(zip(cols, row))
        if line['Bait'] in synonyms:
            line['Bait'] = synonyms[line['Bait']]
        if line['Prey'] in synonyms:
            line['Prey'] = synonyms[line['Prey']]
        bait_prey = frozenset((line['Bait'], line['Prey']))
        line['HCIP_label'] = 1 if bait_prey in HCIP_training_labels else 0
        for col in columns_dtypes:
            line[col] = columns_dtypes[col](line[col])
        features_table.append(line)

In [5]:
# split data by batches & format ready for training
feats = ['Z_binned','WD_binned', 'batch_Z_binned', 'Entropy_binned','Ratio_binned','total_PSMs_binned','ratio_total_PSMs_binned','pep_ratio_binned']

features_table = sorted(features_table, key=lambda x: x['Batch'])
for batch, grp in itt.groupby(features_table, key=lambda x: x['Batch']):
    batch_data=[[(x['Bait'], x['Prey']), np.array([x[f] for f in feats]), x['HCIP_label']] for x in grp]
    pickle.dump(batch_data, open('bioplex_data_batches/{}.pkl'.format(batch), 'wb'))

all_batches = list(set(x['Batch'] for x in features_table))

In [6]:
CP = ComppassPlus(all_batches)

In [7]:
batch_models_dict = CP.compute_models(all_batches, serial=False)
pickle.dump(batch_models_dict, open('bioplex_compassplus_models.pkl', 'wb'))

In [8]:
# bypass training...
batch_models_dict = pickle.load(open('bioplex_compassplus_models.pkl', 'rb'))
CP = ComppassPlus(all_batches, batch_models_dict = batch_models_dict)

In [9]:
HCIP_pred = CP.predict_all()

In [10]:
!cp HCIP_pred_BioPlex_OP16.pkl HCIP_pred_BioPlex_OP16.pkl.back

In [11]:
pickle.dump(HCIP_pred, open('HCIP_pred_BioPlex_OP16.pkl', 'wb'))